<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week2/part1/2_04_Parsear_Procesar_Salida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎯 Output Parsers en LangChain 1.0

**Plataforma:** Google Colab

---

## 📋 Configuración Previa

1. Configura `GOOGLE_API_KEY` en Colab Secrets (🔑)
2. Ejecuta las celdas en orden
3. Espera 3-5 segundos entre llamadas a la API

## 🔑 Celda 1: Configuración de API Key

In [ ]:
from google.colab import userdata
import os
import time


# Obtener la API key desde userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Opcional: Guardarla como variable de entorno
os.environ['GOOGLE_API_KEY'] = api_key

# Verificar que se haya cargado correctamente
print("API Key cargada:", "Sí" if api_key else "No")
print("Primeros caracteres:", api_key[:10] if api_key else "No encontrada")



## 📦 Celda 2: Instalación de Paquetes

In [ ]:
# Instalación de paquetes
!pip install -q -U langchain langchain_community langchain_core langchain-google-genai

print("✅ Instalación completada")

In [ ]:
pip show langchain

## 🤖 Celda 3: Configuración del Modelo Gemini

In [ ]:
# Importar y configurar Gemini
from langchain_google_genai import ChatGoogleGenerativeAI

chat = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,  # Temperature 0 para respuestas más deterministas
    max_retries=2
)

print("✅ Modelo Gemini configurado")

## 📝 Ejemplo 1: CommaSeparatedListOutputParser

Parser para convertir listas separadas por comas en listas de Python.

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 1: CommaSeparatedListOutputParser")
print("="*60)

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Crear el parser
output_parser = CommaSeparatedListOutputParser()

# Ver las instrucciones de formato
format_instructions = output_parser.get_format_instructions()
print("\n📋 Instrucciones de formato:")
print(format_instructions)

# Crear el prompt
prompt = ChatPromptTemplate.from_messages([
    ("user", "{request}\n{format_instructions}")
])

# Crear la cadena (chain) usando el operador |
chain = prompt | chat | output_parser

# Ejecutar
print("\n🤖 Ejecutando consulta...")
time.sleep(3)

try:
    result = chain.invoke({
        "request": "Dame 5 características de los coches americanos",
        "format_instructions": format_instructions
    })

    print("\n✅ Resultado parseado (lista de Python):")
    print(result)
    print(f"\nTipo de dato: {type(result)}")
    print(f"Número de elementos: {len(result)}")

    print("\n📊 Elementos individuales:")
    for i, item in enumerate(result, 1):
        print(f"   {i}. {item}")

except Exception as e:
    print(f"❌ Error: {e}")

## 📅 Ejemplo 2: Parsear Fechas con StrOutputParser

En LangChain 1.0, `DatetimeOutputParser` fue deprecado. Usamos `StrOutputParser` y convertimos manualmente.

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 2: Parsear Fechas como String")
print("="*60)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime

# Parser de string simple
output_parser_str = StrOutputParser()

# Prompt que pide formato específico
prompt_fecha_simple = ChatPromptTemplate.from_messages([
    ("system", "Responde ÚNICAMENTE con la fecha en formato YYYY-MM-DD, sin texto adicional."),
    ("user", "¿En qué fecha fue {evento}?")
])

# Crear cadena
chain_fecha_simple = prompt_fecha_simple | chat | output_parser_str

print("\n🤖 Ejecutando...")
time.sleep(5)

try:
    fecha_str = chain_fecha_simple.invoke({
        "evento": "la declaración de independencia de los EEUU"
    })

    print(f"\n✅ Fecha como string: {fecha_str}")

    # Convertir manualmente a datetime
    fecha_obj = datetime.strptime(fecha_str.strip(), "%Y-%m-%d")

    print(f"\n📅 Fecha convertida: {fecha_obj}")
    print(f"   Año: {fecha_obj.year}")
    print(f"   Mes: {fecha_obj.month}")
    print(f"   Día: {fecha_obj.day}")
    print(f"   Formato: {fecha_obj.strftime('%d/%m/%Y')}")

except ValueError as ve:
    print(f"❌ Error al convertir fecha: {ve}")
    print(f"   Respuesta recibida: {fecha_str}")
except Exception as e:
    print(f"❌ Error: {e}")

## 🔧 Ejemplo 3: Parsear Fechas con PydanticOutputParser

La forma moderna y recomendada en LangChain 1.0.

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 3: Parsear Fechas con Pydantic")
print("="*60)

from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from datetime import datetime

# Definir modelo Pydantic para fechas
class FechaHistorica(BaseModel):
    fecha: datetime = Field(description="Fecha del evento en formato ISO 8601")
    evento: str = Field(description="Nombre del evento histórico")

# Crear parser
fecha_parser = PydanticOutputParser(pydantic_object=FechaHistorica)

# Ver instrucciones
print("\n📋 Instrucciones:")
print(fecha_parser.get_format_instructions()[:300] + "...")

# Crear prompt
prompt_fecha_pydantic = ChatPromptTemplate.from_messages([
    ("system", "Proporciona información histórica en formato JSON."),
    ("user", "{request}\n{format_instructions}")
])

# Crear cadena
chain_fecha_pydantic = prompt_fecha_pydantic | chat | fecha_parser

print("\n🤖 Ejecutando...")
time.sleep(5)

try:
    result = chain_fecha_pydantic.invoke({
        "request": "¿Cuándo fue la llegada del hombre a la Luna?",
        "format_instructions": fecha_parser.get_format_instructions()
    })

    print("\n✅ Resultado:")
    print(f"   Evento: {result.evento}")
    print(f"   Fecha: {result.fecha}")
    print(f"   Año: {result.fecha.year}")
    print(f"   Mes: {result.fecha.month}")
    print(f"   Día: {result.fecha.day}")

except Exception as e:
    print(f"❌ Error: {e}")

## 🎯 Ejemplo 5: PydanticOutputParser (Completo)

Parser con validación de tipos usando Pydantic.

In [ ]:
print("\n" + "="*60)
print("EJEMPLO 5: PydanticOutputParser (Con validación)")
print("="*60)

from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from datetime import datetime
from typing import List

# Definir el modelo Pydantic
class Coche(BaseModel):
    marca: str = Field(description="Marca del carro")
    modelo: str = Field(description="Modelo del carro")
    año: int = Field(description="Año de fabricación")
    caracteristicas: List[str] = Field(description="Lista de características principales")

# Crear el parser
pydantic_parser = PydanticOutputParser(pydantic_object=Coche)

# Ver las instrucciones
print("\n📋 Instrucciones para Pydantic:")
print(pydantic_parser.get_format_instructions()[:300] + "...")

# Crear el prompt
prompt_pydantic = ChatPromptTemplate.from_messages([
    ("system", "Proporciona información detallada sobre carros en formato JSON."),
    ("user", "{request}\n{format_instructions}")
])

# Crear la cadena
chain_pydantic = prompt_pydantic | chat | pydantic_parser

print("\n🤖 Ejecutando con Pydantic...")
time.sleep(5)

try:
    result_pydantic = chain_pydantic.invoke({
        "request": "Dame información sobre el Tesla Model 3",
        "format_instructions": pydantic_parser.get_format_instructions()
    })

    print("\n✅ Resultado con validación Pydantic:")
    print(f"Objeto: {result_pydantic}")
    print(f"\nTipo: {type(result_pydantic)}")
    print(f"\n📊 Acceso a atributos:")
    print(f"   Marca: {result_pydantic.marca}")
    print(f"   Modelo: {result_pydantic.modelo}")
    print(f"   Año: {result_pydantic.año}")
    print(f"   Características:")
    for i, caract in enumerate(result_pydantic.caracteristicas, 1):
        print(f"      {i}. {caract}")

except Exception as e:
    print(f"❌ Error: {e}")

---

## 📚 Referencias

- [LangChain Output Parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)
- [Pydantic Documentation](https://docs.pydantic.dev/)
- [Google AI Studio](https://aistudio.google.com/)

## ⚠️ Notas Importantes

- **Rate Limits**: Gemini gratuito ~15 requests/minuto
- **Esperas**: Los `time.sleep(5)` son necesarios
- **Temperature**: Usa 0 para parseo consistente
- **Migración**: `DatetimeOutputParser` fue deprecado en 1.0

---